View the chopped bounding boxes one by one, select an object and go through the bounding boxes

In [ ]:
from pathlib import Path

images_dir = Path('dataset/icub/Images')
images_part1_chopped_dir = Path('dataset/icub/ImagesCropped')
annotations_dir = Path('dataset/icub/Annotations_refined')
manual_subset_dir = Path('dataset/icub/Images_subset_manual')
manual_annotations_dir = Path('dataset/icub/Annotations_manual')

In [8]:
# the category is book, which is in both chopped and images_dir

category_subpath = 'book/book1/MIX/day5/left'

In [9]:
# get all the images from both directories

images_part1_cropped = sorted(list(images_part1_chopped_dir.joinpath(category_subpath).glob(f'*.jpg')))
images = sorted(list(images_dir.joinpath(category_subpath).glob('*.jpg')))
images_subset_manual = sorted(list(manual_subset_dir.joinpath(category_subpath).glob('*.jpg')))


# remove the images that does not exist in all three directories
images_subset_manual_names = set([f.name for f in images_subset_manual])
images_part1_cropped = [img for img in images_part1_cropped if img.name in images_subset_manual_names]
images = [img for img in images if img.name in images_subset_manual_names]

In [10]:
from lxml import etree

# get the annotations of the images
annotations_xml_files = sorted(list(annotations_dir.joinpath(category_subpath).glob('*.xml')))

annotations_refined = {}

# check every annotations to find bboxes with negative values
# remove the annotation and the corresponding image if there is a negative value
for i, annotation in enumerate(annotations_xml_files):
    tree = etree.parse(str(annotation))
    root = tree.getroot()
    for obj in root.iter('object'):
        bndbox = obj.find('bndbox')

        x_min = int(bndbox.find('xmin').text)
        y_min = int(bndbox.find('ymin').text)
        x_max = int(bndbox.find('xmax').text)
        y_max = int(bndbox.find('ymax').text)

        # check if the bbox is valid
        if ((x_min < 0) or (y_min < 0) or (x_max < 0) or (y_max < 0)):
            continue

        annotations_refined[annotation.name[:-4]] = {
            'x_min': x_min,
            'y_min': y_min,
            'x_max': x_max,
            'y_max': y_max
        }

In [13]:
# get the annotations of the images
annotations_manual_xml_files = sorted(list(manual_annotations_dir.joinpath(category_subpath).glob('*.xml')))

annotations_manual = {}

# check every annotations to find bboxes with negative values
# remove the annotation and the corresponding image if there is a negative value
for i, annotation in enumerate(annotations_manual_xml_files):
    tree = etree.parse(str(annotation))
    root = tree.getroot()
    for obj in root.iter('object'):
        bndbox = obj.find('bndbox')

        x_min = int(bndbox.find('xmin').text)
        y_min = int(bndbox.find('ymin').text)
        x_max = int(bndbox.find('xmax').text)
        y_max = int(bndbox.find('ymax').text)

        # check if the bbox is valid
        if ((x_min < 0) or (y_min < 0) or (x_max < 0) or (y_max < 0)):
            continue
    
        annotations_manual[annotation.name[:-4]] = {
            'x_min': x_min,
            'y_min': y_min,
            'x_max': x_max,
            'y_max': y_max
        }


In [ ]:
import cv2

# create two image streams showing the images from both directories, with the annotations on the images from images list

def show_images(images, images_part1_chopped, images_subset_manual, annotations_refined, annotations_manual):
    for i, image in enumerate(images):
        # read the image
        img = cv2.imread(str(image))
        image_part1_cropped = cv2.imread(str(images_part1_chopped[i]))

        if images_subset_manual:
            image_subset_manual = cv2.imread(str(images_subset_manual[i]))

        # read the annotation
        annotation_refined = annotations_refined.get(str(image.name[:-4]))

        if annotation_refined is not None:
            x_min = annotation_refined['x_min']
            y_min = annotation_refined['y_min']
            x_max = annotation_refined['x_max']
            y_max = annotation_refined['y_max']
            # draw the bbox on the image
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

        if annotations_manual:
            annotation_manual = annotations_manual.get(str(image.name[:-4]))
            if annotation_manual is not None:
                x_min = annotation_manual['x_min']
                y_min = annotation_manual['y_min']
                x_max = annotation_manual['x_max']
                y_max = annotation_manual['y_max']
                # draw the bbox on the image
                cv2.rectangle(image_subset_manual, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.imshow('image_subset_manual', image_subset_manual)
        
        # show the image
        cv2.imshow('image', img)
        cv2.imshow('image_part1_cropped', image_part1_cropped)

        # wait for a key press
        key = cv2.waitKey(0)
        if key == ord('q'):
            break

    cv2.destroyAllWindows()

# show the images
show_images(images, images_part1_cropped, images_subset_manual, annotations_refined, annotations_manual)